In [ ]:
from glob import glob

from PIL import Image


file_paths = []
ellipse_data = []

label_files = glob("../data/FDDB/labels/*.txt")

for filename in label_files:
    with open(filename) as f:
        if "ellipseList" not in filename:
            file_paths += [p.strip("\n") for p in f.readlines()]
        else:
            ellipse_data += [p.strip("\n") for p in f.readlines()]

ellipse_data

In [ ]:
import pandas as pd

In [ ]:
pdf_file_paths = pd.DataFrame({"path": file_paths})
pdf_file_paths["path_idx"] = pdf_file_paths.index

pdf_ellipse_data = pd.DataFrame({"data": ellipse_data})
pdf_ellipse_data["data_idx"] = pdf_ellipse_data.index

In [ ]:
pdf_file_data_mapping = pdf_file_paths.merge(
    pdf_ellipse_data, left_on="path", right_on="data", how="left"
)

In [ ]:
ellipse_dict = {str(k): [] for k in pdf_file_paths["path"]}
ellipse_dict

In [ ]:
for i, r in pdf_file_data_mapping.iterrows():
    data_idx = r["data_idx"]
    num_ellipses = int(ellipse_data[data_idx + 1])
    file_path = r["path"]
    for j in range(data_idx + 2, data_idx + num_ellipses + 2):
        ellipse_dict[file_path].append(
            [float(v) for v in ellipse_data[j].split(" ")[:-1] if len(v) > 0]
        )

In [ ]:
import torch

ellipse_dict["2003/01/14/big/img_1115"]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

img_name = "2002/07/19/big/img_392"

img = Image.open(f"../data/FDDB/images/{img_name}.jpg")
img_array = np.array(img)

In [ ]:
img

In [ ]:
from matplotlib.collections import EllipseCollection

fig, ax = plt.subplots(figsize=(10, 10))
for ed in ellipse_dict[img_name]:
    major_axis_radius, minor_axis_radius, angle, center_x, center_y = ed

    ec = EllipseCollection(
        major_axis_radius * 2,
        minor_axis_radius * 2,
        np.degrees(angle),
        units="xy",
        offsets=(center_x, center_y),
        transOffset=ax.transData,
        facecolors="None",
    )
    ax.add_collection(ec)
ax.imshow(img, cmap="gray")

In [ ]:
from ellipse_rcnn.utils.viz import DetectionPlotter

In [ ]:
major_axis_radius, minor_axis_radius, angle, center_x, center_y = ed
DetectionPlotter.from_ellipses(ed)

In [ ]:
torch.tensor(ed)

In [ ]:
from ellipse_rcnn.utils.data.fddb import FDDB

ds = FDDB("../data/FDDB/")

In [ ]:
image, target = ds[500]
target

In [ ]:
from ellipse_rcnn import EllipseRCNN

model = EllipseRCNN()

In [ ]:
from ellipse_rcnn.utils.data.base import collate_fn

images, targets = collate_fn([(image, target)])

model.forward(images, targets)

In [ ]:
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from ellipse_rcnn.core.model import EllipseRCNNLightning


dataset_loader = DataLoader(ds, batch_size=24, shuffle=True, collate_fn=collate_fn)

pl_model = EllipseRCNNLightning(model)
trainer = Trainer(max_epochs=10, accelerator="auto", log_every_n_steps=1)
trainer.fit(pl_model, dataset_loader)